In [1]:
import tensorflow as tf
from dataset_tf import VoxDataset, DataGenerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
# from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from weighted_mse import WeightedLoss
# from weighted_chamfer_loss import WeightedChamferLoss
import matplotlib.pyplot as plt
import keras
import pickle
import os

In [1]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
tf.__version__

'2.15.0'

In [3]:
dataset = VoxDataset(gender="m")

In [4]:
ids = dataset.get_ids()
len(ids)

67200


3360

In [5]:
len(ids)

3360

In [6]:
len(dataset)

67200

In [7]:
def get_split():
    DATASET_SIZE = len(dataset.get_ids())

    train_size = int(0.6 * DATASET_SIZE)
    val_size = int(0.2 * DATASET_SIZE)
    test_size = DATASET_SIZE - train_size - val_size

    np.random.seed(42)
    np.random.shuffle(ids)

    # Split the indices into training, validation, and test sets
    train_indices = ids[:train_size]
    val_indices = ids[train_size:train_size + val_size]
    test_indices = ids[train_size + val_size:DATASET_SIZE]

    train = VoxDataset(ids=train_indices)
    test = VoxDataset(ids=test_indices)
    val = VoxDataset(ids=val_indices)

    return train, test, val

In [8]:
train, test, val = get_split()

67200


In [9]:
train_ids = train.get_ids()
test_ids = test.get_ids()
val_ids = val.get_ids()

40320
13440
13440


In [10]:
df = test.get_df_()
df.shape

(13440, 8)

In [11]:
df.head(50)

,index,Unnamed: 0.1,Unnamed: 0,id,audio_dir,emb_dir,mesh_dir,gender
0,0,0,0,id00012,./Preprocessed/id00012/00002.mp3,./Preprocessed/id00012/Embeddings/00002.mp3.npy,./Preprocessed/id00012/mesh.ply,m
1,1,1,1,id00012,./Preprocessed/id00012/00032.mp3,./Preprocessed/id00012/Embeddings/00032.mp3.npy,./Preprocessed/id00012/mesh.ply,m
2,2,2,2,id00012,./Preprocessed/id00012/00034.mp3,./Preprocessed/id00012/Embeddings/00034.mp3.npy,./Preprocessed/id00012/mesh.ply,m
3,3,3,3,id00012,./Preprocessed/id00012/00035.mp3,./Preprocessed/id00012/Embeddings/00035.mp3.npy,./Preprocessed/id00012/mesh.ply,m
4,4,4,4,id00012,./Preprocessed/id00012/00037.mp3,./Preprocessed/id00012/Embeddings/00037.mp3.npy,./Preprocessed/id00012/mesh.ply,m
5,5,5,5,id00012,./Preprocessed/id00012/00043.mp3,./Preprocessed/id00012/Embeddings/00043.mp3.npy,./Preprocessed/id00012/mesh.ply,m
6,6,6,6,id00012,./Preprocessed/id00012/00063.mp3,./Preprocessed/id00012/Embeddings/00063.mp3.npy,./Preprocessed/id00012/mesh.ply,m
7,7,7,7,id00012,./Preprocessed/id00012/00073.mp3,./Preprocessed/id00012/Embeddings/00073.mp3.npy,./Preprocessed/id00012/mesh.ply,m
8,8,8,8,id00012,./Preprocessed/id00012/00075.mp3,./Preprocessed/id00012/Embeddings/00075.mp3.npy,./Preprocessed/id00012/mesh.ply,m
9,9,9,9,id00012,./Preprocessed/id00012/00076.mp3,./Preprocessed/id00012/Embeddings/00076.mp3.npy,./Preprocessed/id00012/mesh.ply,m


In [12]:
df['id'].value_counts()

id
id00012    20
id06120    20
id06004    20
id06010    20
id06019    20
           ..
id03016    20
id03018    20
id03029    20
id03039    20
id08891    20
Name: count, Length: 672, dtype: int64

In [13]:
sorted(test_ids)

['id00012',
 'id00015',
 'id00018',
 'id00036',
 'id00049',
 'id00051',
 'id00056',
 'id00084',
 'id00085',
 'id00087',
 'id00089',
 'id00163',
 'id00167',
 'id00169',
 'id00170',
 'id00172',
 'id00173',
 'id00176',
 'id00177',
 'id00184',
 'id00185',
 'id00186',
 'id00192',
 'id00222',
 'id00225',
 'id00255',
 'id00257',
 'id00258',
 'id00266',
 'id00269',
 'id00274',
 'id00296',
 'id00298',
 'id00302',
 'id00309',
 'id00316',
 'id00366',
 'id00389',
 'id00481',
 'id00514',
 'id00518',
 'id00521',
 'id00526',
 'id00528',
 'id00530',
 'id00531',
 'id00545',
 'id00554',
 'id00555',
 'id00586',
 'id00597',
 'id00685',
 'id00699',
 'id00710',
 'id00725',
 'id00732',
 'id00760',
 'id00778',
 'id00781',
 'id00802',
 'id00815',
 'id00852',
 'id00905',
 'id00907',
 'id00918',
 'id00943',
 'id00944',
 'id00960',
 'id00969',
 'id00987',
 'id01043',
 'id01047',
 'id01051',
 'id01052',
 'id01060',
 'id01061',
 'id01101',
 'id01105',
 'id01108',
 'id01111',
 'id01115',
 'id01124',
 'id01157',
 'id

In [14]:
print(len(train))
print(len(test))
print(len(val))

40320
13440
13440


In [15]:
import tensorflow as tf

def AutoEncoder():
    # Define input layer
    input_layer = tf.keras.layers.Input(shape=(384,))
    
    # Encoder layers
    encoder = tf.keras.layers.Dense(384, activation='tanh')(input_layer)

    # encoder = input_layer

    # attention = tf.keras.layers.Attention()([encoder, encoder])
    # weighted_bottleneck = tf.keras.layers.multiply([attention, encoder])

    # attention = tf.keras.layers.Attention()([weighted_bottleneck, weighted_bottleneck])
    # weighted_bottleneck = tf.keras.layers.multiply([attention, weighted_bottleneck])
    
    # attention = tf.keras.layers.Attention()([weighted_bottleneck, weighted_bottleneck])
    # weighted_bottleneck = tf.keras.layers.multiply([attention, weighted_bottleneck])

    # attention = tf.keras.layers.Attention()([weighted_bottleneck, weighted_bottleneck])
    # weighted_bottleneck = tf.keras.layers.multiply([attention, weighted_bottleneck])

    # Bottleneck Decoders 
    decoder = tf.keras.layers.Dense(384, activation='tanh')(encoder)
    decoder = tf.keras.layers.Dense(500, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(768, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(1000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(1536, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(2000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(3072, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(4000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(5000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(12288, activation='tanh')(decoder)
    # decoder = tf.keras.layers.Dense(15000, activation='tanh')(decoder) # layer to comment when training female
    decoder = tf.keras.layers.Dense(15069, activation='tanh')(decoder)

    decoder_output = tf.keras.layers.Dense(15069, activation='tanh')(decoder)
    
    # Define the model
    model = tf.keras.models.Model(inputs=input_layer, outputs=decoder_output)
    
    return model

In [16]:
num_epochs = 100
batch = 1
loss_weight = 1
model_name = "e1_w_ones_w_weightedMSE_w_customL2_14l_lr00001_male_hausdorff"

In [17]:
try:
  os.mkdir(f"./models/{model_name}")
except FileExistsError:
  pass

In [18]:
train = DataGenerator(train, batch_size=batch)
val = DataGenerator(val, batch_size=batch)
test = DataGenerator(test, batch_size=batch)

In [19]:
model = AutoEncoder()


2024-04-26 00:36:11.332890: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-04-26 00:36:11.332908: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-26 00:36:11.332911: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-26 00:36:11.332937: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-26 00:36:11.332951: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
print(len(test[len(test)-1][1]))

1


In [21]:
attention_ids = np.load("./weight_indices.npy")

In [22]:
len(attention_ids)

3540

In [23]:
with open('custom_loss_dict.pkl', 'rb') as file:
    custom_metric = pickle.load(file)

In [24]:
base_loss = tf.keras.losses.MeanSquaredError()

weighted_loss = WeightedLoss(base_loss=base_loss, attention_ids=attention_ids,custom_metric_dict=custom_metric, weight=loss_weight, batch_size=batch)
# weighted_chamfer_loss = WeightedChamferLoss(attention_ids=attention_ids, weight=loss_weight, batch_size=batch)

In [25]:
# for i in tqdm(weights):
#     print(i, end='\r')

In [26]:
# Define the learning rate decay schedule
# learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.001,
#     decay_steps=num_epochs * 3,
#     decay_rate=0.1,
#     staircase=True
# )

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.00001)

# Compile the model with the optimizer
model.compile(optimizer=optimizer, loss=weighted_loss)

early_stopping_val_loss = EarlyStopping(monitor='val_loss', patience=6, verbose=1)
early_stopping_train_loss = EarlyStopping(monitor='loss', patience=3, verbose=1)

# Train the model using model.fit
history = model.fit(train, epochs=num_epochs, batch_size=batch, validation_data=val, 
          callbacks=[early_stopping_train_loss, early_stopping_val_loss]
          )

Epoch 1/100


2024-04-26 00:36:13.752303: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense_26 (Dense)            (None, 384)               147840    
                                                                 
 dense_27 (Dense)            (None, 384)               147840    
                                                                 
 dense_28 (Dense)            (None, 500)               192500    
                                                                 
 dense_29 (Dense)            (None, 768)               384768    
                                                                 
 dense_30 (Dense)            (None, 1000)              769000    
                                                                 
 dense_31 (Dense)            (None, 1536)              1537

In [ ]:
import pickle

with open(f'./models/{model_name}/loss_config.pkl', 'wb') as file:
    pickle.dump(weighted_loss.get_config(), file)

# loss = model.evaluate(test)
# acc = 0

In [ ]:
# print(f"Accuracy: {acc*100:.4f}%\nLoss: {loss:.4f}")

In [ ]:
model.save(f'./models/{model_name}/model.h5')

/Users/naweenk/miniforge3/envs/demo/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
weighted_loss = WeightedLoss(base_loss=base_loss, attention_ids=attention_ids,custom_metric_dict=custom_metric, weight=loss_weight, batch_size=1)

In [ ]:
weighted_loss.get_config()

{'base_loss': <keras.src.losses.MeanSquaredError at 0x37ea02bc0>,
 'attention_ids': array([   0,    1,    2, ..., 5020, 5021, 5022]),
 'weight': 1,
 'batch_size': 1,
 'custom_metric_dict': {'nose_bridge': [3516, 3526],
  'r_eye_lid': [3690, 2265],
  'l_eye_lid': [3856, 809],
  'lip': [3543, 3503],
  'r_lip_bend': [2850, 3798],
  'l_lip_bend': [1735, 3021],
  'r_lip_jaw': [3798, 3406],
  'l_lip_jaw': [3021, 3614],
  'lip_chin': [3503, 3487],
  'orbital_lower': [3710, 3866],
  'oribtal_upper': [3154, 2135],
  'puffer': [3436, 3667],
  'mouth_corner': [2827, 1710],
  'jaw_end': [3406, 3614],
  'ear_end': [856, 288]}}

In [ ]:
model = tf.keras.models.load_model(f"./models/{model_name}/model.h5", 
    custom_objects={"WeightedLoss" : weighted_loss})


In [ ]:
loss = model.evaluate(test)
acc = 0

420/420 [==============================] - 315s 348ms/step - loss: 36.1060


In [ ]:
y_pred = []

In [ ]:
test[419][0].shape

(32, 384)

In [ ]:
model.predict(test[0][0]).shape

1/1 [==============================] - 1s 709ms/step


(32, 15069)

In [ ]:
for i in tqdm(test):
    pred = (model.predict(i[0], verbose=0)
            #  *201.41335
            )
    for j in range(len(pred)):
        y_pred.append([i[1][j], VoxDataset.to_mesh_points(pred[j])])



100%|██████████| 420/420 [00:51<00:00,  8.14it/s]


In [ ]:
len(y_pred)

13440

In [ ]:
import open3d as o3d
import numpy as np

for index, i in enumerate(tqdm(y_pred)):
    vertices = o3d.utility.Vector3dVector(i[1])
    triangles = o3d.utility.Vector3iVector(dataset.get_faces())
    mesh = o3d.geometry.TriangleMesh(vertices, triangles)
    o3d.io.write_triangle_mesh(f"./Test/data_{index}.obj", mesh, 
                            #    write_ascii=False
                               )

100%|██████████| 13440/13440 [10:58<00:00, 20.40it/s]


In [ ]:
test_ids

['id03342',
 'id01598',
 'id00202',
 'id07511',
 'id04875',
 'id00220',
 'id07740',
 'id08513',
 'id05466',
 'id06491',
 'id03907',
 'id06928',
 'id01162',
 'id03283',
 'id07666',
 'id04127',
 'id01093',
 'id01590',
 'id04229',
 'id08283',
 'id01273',
 'id05998',
 'id03719',
 'id04093',
 'id06688',
 'id07547',
 'id03089',
 'id04503',
 'id03546',
 'id03189',
 'id05774',
 'id05408',
 'id06595',
 'id08201',
 'id01344',
 'id06336',
 'id07182',
 'id03146',
 'id08653',
 'id02715',
 'id04107',
 'id04326',
 'id04482',
 'id08219',
 'id02300',
 'id02911',
 'id08749',
 'id07902',
 'id08470',
 'id02343',
 'id02899',
 'id07886',
 'id00266',
 'id01913',
 'id03803',
 'id05039',
 'id07973',
 'id08771',
 'id03070',
 'id08746',
 'id03765',
 'id05997',
 'id05350',
 'id01255',
 'id00311',
 'id06474',
 'id01544',
 'id00096',
 'id04531',
 'id01592',
 'id07863',
 'id00586',
 'id04463',
 'id06252',
 'id00303',
 'id00524',
 'id03945',
 'id08651',
 'id06821',
 'id07679',
 'id06213',
 'id07194',
 'id08246',
 'id

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
# plt.ylim(0, 20)
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
# import numpy as np
# from scipy.spatial import Delaunay
# import trimesh

# points = y_pred[0]

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)

# # Estimate normals for the point cloud
# pcd.estimate_normals()

# # Create a surface mesh from the point cloud using Poisson reconstruction
# mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# # Visualize the mesh (optional)
# o3d.visualization.draw_geometries([mesh])

In [ ]:
# point_cloud = o3d.geometry.PointCloud()
# triangle_mesh = o3d.geometry.TriangleMesh()

# # Define the point cloud data (replace with your actual data)

# point_cloud.points = o3d.utility.Vector3dVector(i)

# # Define the triangle data (replace with your actual data)
# triangles = dataset.get_faces()  # Example triangle indices
# triangle_mesh.triangles = triangles

# # Combine the point cloud and triangle mesh into a single geometry (optional)
# combined_geometry = point_cloud + triangle_mesh  # Combine point cloud and mesh

# # Write the geometry to a PLY file
# o3d.io.write_point_cloud("point_cloud_and_mesh.ply", combined_geometry)

In [ ]:
test[0][0].shape